In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
os.getcwd()
os.chdir( "C:/Users/Miray/Desktop")
os.getcwd()
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, mean_squared_error
from sklearn import tree 
from sklearn.naive_bayes import GaussianNB
from warnings import filterwarnings
filterwarnings('ignore')

**Bu notebook'ta Target 90 değişkeni Class'a dönüştürülmüştür ve Naive Bayes (Multinomial)  uygulanmıştır.
Ön işleme olarak scaler edilmiştir.**

In [2]:
odv=pd.read_csv("odev.csv")
df=odv.copy()
df.loc[df[df["ind_420"] == "?"].index] = np.nan 
df["ind_422"] = df["ind_422"].astype('float64')
df["ind_420"] = df["ind_420"].astype('float64')
df.dropna()
for i in df.select_dtypes(include=["object"]).columns:
    print(i,": ",df[i].value_counts().count())

ind_109 :  2


In [3]:
df.dropna()
df.dropna(axis=0,inplace=True)
df.isnull().values.any()

False

In [4]:
columns = ['20_target','50_target']
df.drop(columns, inplace=True, axis=1)


In [5]:
target_90=[]

for i in df['90_target']: 
    
    if i == 0 :
    
        target_90.append('notr')
    
    elif i < 0: 
    
        target_90.append('negatif')
        
    else :
        
        target_90.append('pozitif')
    
df['target_90'] = target_90

# Target 90 üç sınıfa dönüştürüldü.

In [6]:
columns = ['90_target']
df.drop(columns, inplace=True, axis=1)


In [7]:
df.head()

,ind_5,ind_6,ind_8,ind_9,ind_10,ind_12,ind_13,ind_14,ind_15,ind_16,...,ind_412,ind_414,ind_416,ind_418,ind_420,ind_422,ind_424,ind_426,ind_428,target_90
0,19.0,17.0,100.0,85.714286,14.285714,72.363515,60.808814,23.80,17.62,11.73,...,0.002371,2.0,-49.6,-54.0,-152.0,-353.0,1.0,0.498547,0.701906,pozitif
1,24.0,19.0,100.0,78.571429,21.428571,74.275883,64.366798,11.45,18.16,12.22,...,0.003074,0.8,-55.6,-60.0,-158.0,-359.0,1.0,0.537088,0.690833,pozitif
2,30.0,24.0,100.0,71.428571,28.571429,75.140402,65.915803,8.75,17.86,12.28,...,0.003561,2.7,-58.4,-60.0,-160.0,-362.0,1.0,0.615169,0.693040,negatif
3,37.0,30.0,100.0,64.285714,35.714286,76.677846,68.584234,7.80,14.76,12.61,...,0.004366,0.9,-61.8,-65.0,-166.0,-367.0,1.0,0.661517,0.673418,negatif
4,41.0,37.0,100.0,57.142857,42.857143,81.603007,76.455495,14.90,11.92,14.25,...,0.004198,0.1,-79.8,-86.0,-186.0,-388.0,1.0,0.747204,0.700522,negatif


In [28]:
df["target_90"].value_counts()

negatif    2810
pozitif    2244
notr         72
Name: target_90, dtype: int64

In [8]:
dms=pd.get_dummies(df[['ind_109']])
dms.head()
y=df["target_90"]
X_=df.drop(["ind_109","target_90"],axis=1).astype("float64")
# X'in alacağı değerlerden kategorikleri ve hedef değişkeni sildik.
X_.head()
X=pd.concat([X_,dms[["ind_109_GREEN"]]],axis=1) #dummy haliyle concat yaparak bağımsız değişken dataframe'i oluşturduk.
X.head()

,ind_5,ind_6,ind_8,ind_9,ind_10,ind_12,ind_13,ind_14,ind_15,ind_16,...,ind_412,ind_414,ind_416,ind_418,ind_420,ind_422,ind_424,ind_426,ind_428,ind_109_GREEN
0,19.0,17.0,100.0,85.714286,14.285714,72.363515,60.808814,23.80,17.62,11.73,...,0.002371,2.0,-49.6,-54.0,-152.0,-353.0,1.0,0.498547,0.701906,1
1,24.0,19.0,100.0,78.571429,21.428571,74.275883,64.366798,11.45,18.16,12.22,...,0.003074,0.8,-55.6,-60.0,-158.0,-359.0,1.0,0.537088,0.690833,1
2,30.0,24.0,100.0,71.428571,28.571429,75.140402,65.915803,8.75,17.86,12.28,...,0.003561,2.7,-58.4,-60.0,-160.0,-362.0,1.0,0.615169,0.693040,1
3,37.0,30.0,100.0,64.285714,35.714286,76.677846,68.584234,7.80,14.76,12.61,...,0.004366,0.9,-61.8,-65.0,-166.0,-367.0,1.0,0.661517,0.673418,1
4,41.0,37.0,100.0,57.142857,42.857143,81.603007,76.455495,14.90,11.92,14.25,...,0.004198,0.1,-79.8,-86.0,-186.0,-388.0,1.0,0.747204,0.700522,1


In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()
scaler.fit(X)
scaler.transform(X)

array([[0.75      , 0.74      , 1.        , ..., 0.49854651, 0.78773616,
        1.        ],
       [0.775     , 0.75      , 1.        , ..., 0.53708791, 0.77362977,
        1.        ],
       [0.805     , 0.775     , 1.        , ..., 0.61516854, 0.77644125,
        1.        ],
       ...,
       [0.75      , 0.765     , 0.42857143, ..., 0.63210702, 0.59646706,
        1.        ],
       [0.75      , 0.75      , 1.        , ..., 0.73300971, 0.64719907,
        1.        ],
       [0.75      , 0.75      , 0.92857143, ..., 0.6875    , 0.65019551,
        1.        ]])

In [15]:
X=scaler.transform(X)

In [16]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.30, random_state=42,shuffle=False)

In [17]:
from sklearn.naive_bayes import GaussianNB

In [18]:
from sklearn.naive_bayes import MultinomialNB

In [19]:
mnb=MultinomialNB()
mnb_model=mnb.fit(X_train,y_train)
mnb_model

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
y_pred=mnb_model.predict(X_test)
y_pred[0:10]

array(['negatif', 'negatif', 'negatif', 'negatif', 'negatif', 'negatif',
       'negatif', 'negatif', 'negatif', 'negatif'], dtype='<U7')

In [21]:
accuracy_score(y_pred,y_test)

0.5435630689206762

In [32]:
confusion_matrix(y_test,y_pred) 

array([[652,  10, 177],
       [  0,   0,   0],
       [512,   3, 184]], dtype=int64)

In [31]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

     negatif       0.56      0.78      0.65       839
        notr       0.00      0.00      0.00         0
     pozitif       0.51      0.26      0.35       699

    accuracy                           0.54      1538
   macro avg       0.36      0.35      0.33      1538
weighted avg       0.54      0.54      0.51      1538



In [24]:
mnb_model.predict_proba(X_test)[0:10]

array([[0.65218101, 0.02023399, 0.327585  ],
       [0.65575367, 0.02262715, 0.32161919],
       [0.67412653, 0.02247196, 0.30340151],
       [0.67022289, 0.01945962, 0.31031749],
       [0.67154867, 0.01871264, 0.30973869],
       [0.70635432, 0.01159993, 0.28204575],
       [0.70094796, 0.01087048, 0.28818156],
       [0.69809446, 0.00962841, 0.29227713],
       [0.67116889, 0.0096638 , 0.31916731],
       [0.67219865, 0.01088685, 0.3169145 ]])